### Examples of differet boosting algorithms (new random forest)
1. General usage
2. Analysis
3. Improvements

In [2]:
import pandas as pd
import numpy as np
import lightgbm
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# 
raw_df = pd.read_excel("http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx")

raw_df = raw_df.dropna()

# drop not needed stuff 
raw_df.drop(columns=['StockCode', 'Description'], inplace=True)

raw_df['Quantity'] = raw_df['Quantity'].apply(lambda x: int(x))
raw_df['CustomerID'] = raw_df['CustomerID'].apply(lambda x: int(x))
raw_df['UnitPrice'] = raw_df['UnitPrice'].apply(lambda x: float(x))
raw_df = raw_df.drop(raw_df[raw_df.Quantity <= 0].index)
raw_df['Transaction'] = raw_df["UnitPrice"] * raw_df["Quantity"]

lastDate = raw_df['InvoiceDate'].iloc[-1]
raw_df = raw_df.groupby('CustomerID').agg({'InvoiceDate': lambda x: (lastDate - x.max()).days, 
                                           'InvoiceNo': lambda x: len(x), 
                                           'Transaction': lambda x: x.sum()})
raw_df['InvoiceDate'] = raw_df['InvoiceDate'].astype(int)
raw_df.rename(columns={'InvoiceDate': 'recency', 
                         'InvoiceNo': 'frequency', 
                         'Transaction': 'monetary'}, inplace=True)

raw_df.head()

,recency,frequency,monetary
CustomerID,,,
12346,325,1,77183.60
12347,1,182,4310.00
12348,74,31,1797.24
12349,18,73,1757.55
12350,309,17,334.40


In [23]:
# Adaptive boosting -> combining stmps into one regressor/classifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

# prepare data
train, test = train_test_split(raw_df)
train_target = train['monetary']
train.drop(columns=['monetary'], inplace=True)
test_target = test['monetary']
test.drop(columns=['monetary'], inplace=True)

# Fit regression model
simple = DecisionTreeRegressor(max_depth=4)

boosted = AdaBoostRegressor(DecisionTreeRegressor(max_depth=4), n_estimators=300)

simple.fit(train, train_target)
boosted.fit(train, train_target)

# Predict
simple_predictor = simple.predict(test)
boosted_predictor = boosted.predict(test)

from sklearn.metrics import mean_squared_error

simple_mse = mean_squared_error(list(test_target), simple_predictor)
boosted_mse = mean_squared_error(list(test_target), boosted_predictor)

print("simple: {} vs boosted: {}".format(simple_mse, boosted_mse))
print(simple_mse - boosted_mse)

simple: 100790850.14594287 vs boosted: 204600581.69923657
-103809731.5532937


/home/alisa/projects/ml-showcase/env/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
